# Base de datos MongoDB

## Lectura de datos y desnormalizacion

In [1]:
import pandas as pd

df_dificultades = pd.read_csv("./data/dificultades.csv",encoding='utf-8');
df_encadenamientos = pd.read_csv("./data/tipos_encadenamiento.csv", encoding='utf-8');
df_escaladores = pd.read_csv("./data/escaladores_lite_2017.csv", encoding='utf-8');
df_ascensos = pd.read_csv("./data/ascensos_lite_2017.csv", encoding='utf-8');

# 'merge' del tipo de encadenamiento en el data frame de ascensos
df_ascensos = pd.merge(df_ascensos, df_encadenamientos, on = ['id_tipo_encadenamiento'], how = 'inner')
df_ascensos = df_ascensos[['id_escalador','id_dificultad', 'tipo_encadenamiento', 'nombre_via', 'risco', 'sector', 'pais']];

#df_ascensos = pd.merge(df_ascensos,df_escaladores, on = ['id_escalador'], how = 'inner');
#df_ascensos = pd.merge(df_ascensos.df_dificultades, on = ['id_dificultad'], how = 'inner');
#
## Renombrar campos por claridad (el merge auto-asigna nombres a campos repetidos)
#df_ascensos = df_ascensos.rename(
#    columns={
#        'nombre': 'nombre_escalador',
#        'ciudad':'ciudad_escalador',
#        'pais_y': 'pais_escalador',
#        'comienzo': 'anio_comienzo',
#        'pais_x': 'pais_risco'});
#
## Filtro las columnas de ascensos dejando solo aquellas que me interesan
#columnas = ['id_escalador','nombre_escalador','pais_escalador',
#            'id_dificultad', 'grado_frances', 'tipo_encadenamiento',
#            'nombre_via', 'risco', 'pais_risco']
#
#df_ascensos = df_ascensos[columnas]

## Conexión a Mongo

In [2]:
import pymongo
from pymongo import MongoClient

from pymongo import MongoClient
client = MongoClient('localhost', 27017)

client.drop_database("practica_8anu")

db = client.practica_8anu

## Creacion de collecciones MongoDB

Las preguntas planteadas van en dos direcciones:
- Dados los escaladores, preguntas sobre sus ascensos.
- Preguntas sobre los ascensos en si mismos.

MongoDB no ofrece la posibilidad de hacer Joins de collecciones. Para resolver esas dos direcciones, me viene bien plantear dos colecciones con la siguiente información:

** TO-DO **

In [3]:
df_escaladores.head()

,id_escalador,nombre,sexo,fecha_nacimiento,ciudad,pais,comienzo
0,5,Joe McLoughlin,Hombre,1969-05-07,North Attleboro,USA,1991
1,10,Jens Larssen,Hombre,1965-06-22,Göteborg,SWE,1992
2,28,Knut Rokne,Hombre,1972-03-27,Calgary,CAN,1988
3,35,Jason Kester,Hombre,1971-08-12,portland,USA,1992
4,38,Alan Cassidy,Hombre,1982-12-10,Glasgow,GBR,1993


In [4]:
import json

def inserta_escalador(escalador_json):
    escalador_json['_id']=escalador_json['id_escalador']
    del escalador_json['id_escalador']
    db.escaladores.insert_one(escalador_json)

    
json_string = df_escaladores.to_json(orient = 'records')
lista_json = json.loads(json_string)

# añade los escaladores a la colección    
for escalador_json in lista_json :
    inserta_escalador(escalador_json)

In [5]:
df_ascensos.head(2)

,id_escalador,id_dificultad,tipo_encadenamiento,nombre_via,risco,sector,pais
0,21208,33,Onsight,B52,GöSSLERWAND,ZIG ZAG,AUT
1,58382,25,Onsight,EL CANTO DE LA HORMIGA,MALLORCA,CALA LLAMP,ESP


In [6]:

def inserta_ascenso(ascenso_json):
    #registramos el ascenso en la collección de la bb.dd.
    insert_result = db.ascensos.insert_one(ascenso_json)
    
    #suma uno al contador de ascensos del escalador y actualiza su coleccion de ascensos 
    db.escaladores.find_and_modify(query = {"_id" : ascenso_json['id_escalador']},
                                   update ={ "$inc": { "numero_ascensos": 1 },
                                             "$push": {'ascensos' : insert_result.inserted_id}
                                           })
    


json_string = df_ascensos.to_json(orient = 'records')
lista_json = json.loads(json_string)

# añade los escaladores a la colección    
for ascenso_json in lista_json :
    inserta_ascenso(ascenso_json)

## Respuestas a las preguntas

### 1.a) Los 10 escaladores (hombres) más activos (orden auxiliar por Id)

In [7]:
result = db.escaladores.find(
    { "sexo" : "Hombre" },
    {
    "_id": 1,
    "nombre": 1,
    "sexo": 1,
    "pais": 1,
    "numero_ascensos": 1}).limit(10).sort([("numero_ascensos", -1), ("_id", -1)])

pd.DataFrame(list(result))

,_id,nombre,numero_ascensos,pais,sexo
0,50884,Christopher Leonetti,47,USA,Hombre
1,20095,Matthias Schuster,34,DEU,Hombre
2,66250,Kuba Kaminski,33,POL,Hombre
3,44646,Mateusz Mikosik,25,POL,Hombre
4,20384,Clemens Kurth,23,DEU,Hombre
5,9171,Laurenz Trawnicek,22,AUT,Hombre
6,46622,Raúl Crespo,20,ESP,Hombre
7,23550,Moritz Perwitzschky,20,DEU,Hombre
8,42086,Marcin Opozda,18,POL,Hombre
9,45854,Andrea Roasio,17,ITA,Hombre


### 1.b) Los 10 escaladoras (mujeres) más activas (orden auxiliar por Id)

In [8]:
result = db.escaladores.find(
    { "sexo" : "Mujer" },
    {
    "_id": 1,
    "nombre": 1,
    "sexo": 1,
    "pais": 1,
    "numero_ascensos": 1}).sort([("numero_ascensos", -1), ("_id", -1)]).limit(10)

pd.DataFrame(list(result))

,_id,nombre,numero_ascensos,pais,sexo
0,62807,Claudia Solis,16,MEX,Mujer
1,65502,Ksenia Targosz,15,POL,Mujer
2,53983,Daniela Bärtschi,15,CHE,Mujer
3,60268,Annemarie Van den Berg,12,NLD,Mujer
4,44050,Betka Galicic,11,SVN,Mujer
5,61473,Patricia Schanne,10,CHE,Mujer
6,54695,Elfi Hasler,10,AUT,Mujer
7,48798,Ba Lu,10,POL,Mujer
8,41419,Kasia Latuszek,10,POL,Mujer
9,37559,Magdalena Trzemzalska,10,POL,Mujer


### 2. Lista de los 10 ascensos "On sight" de la escaladora más activa en orden decreciente de dificultad (y por nombre de via ascendete)

### 3. Dificultad media de los ascensos del escalador más activo

### 4.a) Los 10 ascensos mas dificiles

### 4.b) Los 10 ascensos mas dificiles a vista (On sight)

### 5.a) Grado medio y maximo de los ascensos en España de los 10 escaladores NO ESPAÑOLES con mas ascensos en España

### 5.b) Grado medio y maximo de los ascensos en España de los 10 escaladores ESPAÑOLES con mas ascensos en España

### 6.a) Dificultad media y maxima de los ascensos NO "Top Rope" de los escaladores con menos de 3 años de experiencia

### 6.b) Dificultad media y maxima de los ascensos NO "Top Rope" de los escaladores con entre 10 y 30 años de experiencia

### 7. Los 10 riscos españoles (o zonas) con mas ascensos por orden decreciente de numero de ascensos

### 8.a) Los 10 sectores españoles con mayor nivel de difcultad media de ascensos ordenadas por orden decreciente de dificultad y por numero de ascensos decreciente

### 8.b) Las 10 sectores españoles con menor nivel de dificultad medio de ascensos ordenadas por orden creciente de dificultad y por numero de ascensos decreciente